# Frame Extraction and Other Video Processing Scripts

In [ ]:
# !pip install azure-storage-blob tensorflow tensorflow-io av --quiet
!# pip install azure-identity --quiet

try:
  %tensorflow_version 2.x
except Exception:
  pass

!pip install tensorflow-io

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 24.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/harvard.dce.nasa.cloud2cloud

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1DZ81MWvJaMALBYDX54JddfVO9g-pLLu-/harvard.dce.nasa.cloud2cloud


In [ ]:
from google.colab import userdata

account_name = userdata.get('storage_account_name')
account_key = userdata.get('storage_account_key')
container_name = userdata.get('blob_container_name')

In [ ]:
# Set-up environment variable for TFIO connection
import os
import tensorflow as tf
import tensorflow_io as tfio
import pandas as pd

os.environ['TF_AZURE_STORAGE_KEY'] = account_key

output_foldername = "processed_v1"
pathname = f'az://{account_name}/{container_name}/{output_foldername}'

# list files and directories in container named ''
current_dirs_azure = tf.io.gfile.listdir(f'az://{account_name}/{container_name}')

# create dir in azure storage container
if pathname in current_dirs_azure:
  print("path already exists")
else:
  tf.io.gfile.mkdir(pathname)
  print("directory created")

directory created in Azure:  az://nasacloud/nasablob/processed_v1


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import os
import io
import cv2
import tensorflow as tf
import tensorflow_io as tfio
#import av
import time
import shutil
import datetime

In [ ]:
# def delete_tmp_file(file_path, type):
#     """
#     Deletes the tmp file from drive.
#     Args:
#       file_path (str): path to the file to be deleted.
#     """
#     if type == "file":
#       if os.path.exists(file_path):
#           os.remove(file_path)
#       else:
#         print(f"File not found: {file_path}")
#     elif type == "dir":
#       if os.path.exists(file_path):
#           shutil.rmtree(file_path)
#       else:
#         print(f"File not found: {file_path}")


# def download_blob_to_temp(video_path, blob_client):
#     """
#     Download a blob from azure to tmp dir in google dir.
#     Args:
#       video_path (str): path to the video blob.
#     Returns:
#       (str): path to tmp local video file
#     """
#     local_video_path = f'/tmp/{os.path.basename(video_path)}'
#     with open(local_video_path, "wb") as file:
#         data = blob_client.download_blob().readall()
#         file.write(data)
#     return local_video_path


def upload_frames_to_blob(frames_folder, blob_dir, container_client):
    """
    Upload extracted frames from drive to azure blob.
    Args:
        frames_folder (str): path to folder holding the frames
        blob_dir (str): blob where frames are to be uploaded
        container_client (azure.storage.blob.ContainerClient): azure blob storage container client
    Returns:
      None
    """
    for frame in os.listdir(frames_folder):
        frame_path = os.path.join(frames_folder, frame)
        blob_name = f"{blob_dir}/{frame}"
        blob_client = container_client.get_blob_client(blob_name)
        with open(frame_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)

def extract_frames(video_path, frame_rate):
    """
    Extract frames from video at a specified frame rate per second and upload them to blob storage.

    Args:
      video_path (str): path to the video blob in azure
      frame_rate (int): no of frames per second

    Returns:
      (string): o/p folder in the azure blob where frames are uploaded.
    """

    # blob_client = container_client.get_blob_client(video_path)
    # blob_dir = f"{os.path.splitext(video_path)[0]}_frames"
    # print(blob_dir)
    # blob_list = container_client.list_blobs(name_starts_with=blob_dir)
    # print(blob_list)
    # if any(blob_list):
    #     print("blob already exists, skipping frame extraction.")
    #     return f"skipped {video_path}"

    # local tmp storage
    #local_video_path = download_blob_to_temp(video_path, blob_client)
    local_video_path = f'/tmp/{os.path.basename(video_path)}'
    filename = os.path.basename(local_video_path)
    date = filename[:6]
    start_time_str = filename[7:13]
    video_start_time = datetime.datetime.strptime(f"20{date} {start_time_str}", "%Y%m%d %H%M%S")
    #folder_name = f"/tmp/{os.path.join(os.path.dirname(video_path), os.path.splitext(filename)[0] + '_frames')}"

    # # create a folder to store extracted frames
    # if not os.path.exists(folder_name):
    #     os.makedirs(folder_name)

    pathname = f'az://{account_name}/{container_name}/{video_path}'

    with tf.io.gfile.GFile(pathname, mode='r') as r:
      r.read()


    # cap = cv2.VideoCapture(local_video_path)
    # if not cap.isOpened():
    #     print("Error during video load.")
    #     return

    # fps = cap.get(cv2.CAP_PROP_FPS)
    # total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # video_duration = total_frames / fps

    # frame_interval = int(fps / frame_rate)

    # # extract frames
    # count = 0
    # for frame_number in range(0, total_frames, frame_interval):
    #     cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    #     ret, frame = cap.read()
    #     if not ret:
    #         print(f"Error: Unable to read frame {frame_number}.")
    #         continue

    #     frame_time = video_start_time + datetime.timedelta(seconds=frame_number / fps)
    #     img_filename = os.path.join(folder_name, f"{frame_time.strftime('%Y%m%d_%H%M%S')}_frame_{frame_number}.jpg")
    #     cv2.imwrite(img_filename, frame)
    #     count += 1

    # cap.release()

    # # upload extracted frames to azure blob and return blob directory
    # upload_frames_to_blob(folder_name, blob_dir, container_client)

    # # delete tmp files
    # delete_tmp_file(local_video_path, "file")
    # delete_tmp_file(folder_name, "dir")

    return blob_dir

In [ ]:
video_path = "20170418/170418_175706_183328.avi"
frame_rate = 1
output_folder = extract_frames(video_path, frame_rate)
print(f"Frames saved to {output_folder} in Azure Blob Storage")

In [ ]:
frame_rate = 1
blob_list = container_client.list_blobs(name_starts_with="20170422/")
for blob in blob_list:
    if blob.name.endswith('.avi'):
      print(f"Name: {blob.name}")
      output_folder = extract_frames(blob.name, 1)
      print(f"Frames saved to {output_folder} in Azure Blob Storage")

Name: 20170422/170422_203040_210703.avi
20170422/170422_203040_210703_frames
<iterator object azure.core.paging.ItemPaged at 0x7e64971dd9c0>
Frames saved to 20170422/170422_203040_210703_frames in Azure Blob Storage
Name: 20170422/170422_210704_212527.avi
20170422/170422_210704_212527_frames
<iterator object azure.core.paging.ItemPaged at 0x7e64971ddd80>
Frames saved to 20170422/170422_210704_212527_frames in Azure Blob Storage
Name: 20170422/170422_212816_215538.avi
20170422/170422_212816_215538_frames
<iterator object azure.core.paging.ItemPaged at 0x7e64971dfa60>
